In [17]:
import pandas as pd
import psycopg2 as psy
from psycopg2 import Error

import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

DATABASE_URL = os.getenv('DATABASE_URL')

In [3]:
def connection_db() -> psy.extensions.connection:
    try:
        conn = psy.connect(DATABASE_URL)
        return conn
    except (Exception, Error) as e:
        print('Error while connecting to PostgreSQL', e)

In [39]:
try:
    conn = connection_db()
    cursor = conn.cursor()
    cursor.execute(
        """
            select 
                r.*,
                p.stop, p.lap as pit_stop_lap, p.time as pit_stop_time, p.duration, p.milliseconds as pit_stop_milliseconds
            from results r
            LEFT JOIN pit_stops p ON r.raceid = p.raceid;
        """
    )
    
    records = cursor.fetchall()
    records_data = pd.DataFrame(records)

    columns = []
    for column in cursor.description:
        columns.append(column[0])

    records_data.columns = columns

    display(records_data)
except Exception as e:
    print(e)
finally:
    if (conn):
        cursor.close()
        conn.close()
        print("PostgreSQL connection is closed")

,resultid,raceid,driverid,constructorid,number,grid,position,positiontext,positionorder,points,...,fastestlap,rank,fastestlaptime,fastestlapspeed,statusid,stop,pit_stop_lap,pit_stop_time,duration,pit_stop_milliseconds
0,23540,841,39,164,22.0,0.0,None,F,24.0,0.0,...,3,0,None,252.68,81,1.0,1.0,17:05:23,26.898,26898.0
1,23539,841,24,164,23.0,0.0,None,F,23.0,0.0,...,3,0,None,252.698,81,1.0,1.0,17:05:23,26.898,26898.0
2,20800,841,815,15,17.0,13.0,None,D,22.0,0.0,...,39,6,1:29.962,212.209,2,1.0,1.0,17:05:23,26.898,26898.0
3,20799,841,155,15,16.0,9.0,None,D,21.0,0.0,...,51,9,1:30.384,211.218,2,1.0,1.0,17:05:23,26.898,26898.0
4,20798,841,813,3,12.0,15.0,None,R,20.0,0.0,...,7,18,1:34.102,202.873,7,1.0,1.0,17:05:23,26.898,26898.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235835,4077,210,55,22,7.0,14.0,5.0,5,5.0,2.0,...,44,9,None,205.034,11,NaN,NaN,None,None,NaN
235836,4076,210,21,17,12.0,6.0,4.0,4,4.0,3.0,...,43,5,None,213.651,1,NaN,NaN,None,None,NaN
235837,4075,210,56,6,6.0,9.0,3.0,3,3.0,4.0,...,44,5,None,215.702,1,NaN,NaN,None,None,NaN
235838,4074,210,30,6,5.0,3.0,2.0,2,2.0,6.0,...,43,3,None,214.946,1,NaN,NaN,None,None,NaN


PostgreSQL connection is closed


In [40]:
records_data.dtypes

resultid                   int64
raceid                     int64
driverid                   int64
constructorid              int64
number                    object
grid                      object
position                  object
positiontext              object
positionorder             object
points                    object
laps                      object
time                      object
milliseconds               int64
fastestlap                 int64
rank                       int64
fastestlaptime            object
fastestlapspeed           object
statusid                   int64
stop                     float64
pit_stop_lap             float64
pit_stop_time             object
duration                  object
pit_stop_milliseconds    float64
dtype: object

In [41]:
records_data['fastestlapspeed'] = pd.to_numeric(records_data['fastestlapspeed'])
records_data[['raceid', 'resultid', 'driverid', 'constructorid', 'position', 'number']] = records_data[['raceid', 'resultid', 'driverid', 'constructorid', 'position', 'number']].astype('object')
#records_data[['stop', 'pit_stop_lap', 'pit_stop_milliseconds']] = records_data[['stop', 'pit_stop_lap', 'pit_stop_milliseconds']].astype(int)

#records_data['status'] = records_data['status'].astype('category').cat.codes

#records_data.drop('resultid', axis=1, inplace=True)

records_data.dtypes

resultid                  object
raceid                    object
driverid                  object
constructorid             object
number                    object
grid                      object
position                  object
positiontext              object
positionorder             object
points                    object
laps                      object
time                      object
milliseconds               int64
fastestlap                 int64
rank                       int64
fastestlaptime            object
fastestlapspeed          float64
statusid                   int64
stop                     float64
pit_stop_lap             float64
pit_stop_time             object
duration                  object
pit_stop_milliseconds    float64
dtype: object

In [42]:
records_data.isnull().sum()

resultid                      0
raceid                        0
driverid                      0
constructorid                 0
number                        6
grid                          0
position                  44452
positiontext                  0
positionorder                 0
points                        0
laps                          0
time                     122337
milliseconds                  0
fastestlap                    0
rank                          0
fastestlaptime            25796
fastestlapspeed               0
statusid                      0
stop                      20799
pit_stop_lap              20799
pit_stop_time             20799
duration                  20799
pit_stop_milliseconds     20799
dtype: int64

In [43]:
from sklearn.impute import IterativeImputer

In [44]:
temp_df = records_data.select_dtypes(exclude=['object'])

imputer = IterativeImputer(min_value=0, max_iter=30, imputation_order='roman', random_state=1)
imputed_data = imputer.fit_transform(temp_df)

temp_df_imputed = pd.DataFrame(imputed_data, columns=temp_df.columns)
temp_df_imputed.isnull().sum()

milliseconds             0
fastestlap               0
rank                     0
fastestlapspeed          0
statusid                 0
stop                     0
pit_stop_lap             0
pit_stop_milliseconds    0
dtype: int64

In [45]:
records_data.describe()

,milliseconds,fastestlap,rank,fastestlapspeed,statusid,stop,pit_stop_lap,pit_stop_milliseconds
count,2.358400e+05,235840.000000,235840.000000,235840.000000,235840.000000,215041.000000,215041.000000,2.150410e+05
mean,6.054144e+06,42.970628,10.550827,203.285475,10.365477,1.764705,25.329416,7.333744e+04
std,1.409499e+06,17.445171,6.161306,23.061830,20.822461,0.915131,14.673769,2.731860e+05
min,2.070710e+05,0.000000,0.000000,34.401000,1.000000,1.000000,1.000000,1.289700e+04
25%,5.434503e+06,35.000000,5.000000,190.949000,1.000000,1.000000,13.000000,2.188000e+04
50%,5.926470e+06,46.000000,11.000000,202.636000,3.000000,2.000000,25.000000,2.352400e+04
75%,6.606250e+06,54.000000,15.000000,216.479000,11.000000,2.000000,36.000000,2.610900e+04
max,1.509054e+07,158.000000,32.000000,305.950000,141.000000,6.000000,78.000000,3.069017e+06


In [46]:
temp_df.describe()

,milliseconds,fastestlap,rank,fastestlapspeed,statusid,stop,pit_stop_lap,pit_stop_milliseconds
count,2.358400e+05,235840.000000,235840.000000,235840.000000,235840.000000,215041.000000,215041.000000,2.150410e+05
mean,6.054144e+06,42.970628,10.550827,203.285475,10.365477,1.764705,25.329416,7.333744e+04
std,1.409499e+06,17.445171,6.161306,23.061830,20.822461,0.915131,14.673769,2.731860e+05
min,2.070710e+05,0.000000,0.000000,34.401000,1.000000,1.000000,1.000000,1.289700e+04
25%,5.434503e+06,35.000000,5.000000,190.949000,1.000000,1.000000,13.000000,2.188000e+04
50%,5.926470e+06,46.000000,11.000000,202.636000,3.000000,2.000000,25.000000,2.352400e+04
75%,6.606250e+06,54.000000,15.000000,216.479000,11.000000,2.000000,36.000000,2.610900e+04
max,1.509054e+07,158.000000,32.000000,305.950000,141.000000,6.000000,78.000000,3.069017e+06


In [47]:
records_data[temp_df.columns] = temp_df_imputed
records_data.isnull().sum()

resultid                      0
raceid                        0
driverid                      0
constructorid                 0
number                        6
grid                          0
position                  44452
positiontext                  0
positionorder                 0
points                        0
laps                          0
time                     122337
milliseconds                  0
fastestlap                    0
rank                          0
fastestlaptime            25796
fastestlapspeed               0
statusid                      0
stop                          0
pit_stop_lap                  0
pit_stop_time             20799
duration                  20799
pit_stop_milliseconds         0
dtype: int64

In [48]:
records_data.dtypes

resultid                  object
raceid                    object
driverid                  object
constructorid             object
number                    object
grid                      object
position                  object
positiontext              object
positionorder             object
points                    object
laps                      object
time                      object
milliseconds             float64
fastestlap               float64
rank                     float64
fastestlaptime            object
fastestlapspeed          float64
statusid                 float64
stop                     float64
pit_stop_lap             float64
pit_stop_time             object
duration                  object
pit_stop_milliseconds    float64
dtype: object

In [49]:
records_data['milliseconds'] = round(records_data['milliseconds'], 0).astype(int)
records_data['fastestlap'] = records_data['fastestlap'].astype(int)
records_data['rank'] = records_data['rank'].astype(int)
records_data['fastestlapspeed'] = round(records_data['fastestlapspeed'], 3)
records_data[['stop', 'pit_stop_lap', 'pit_stop_milliseconds']] = records_data[['stop', 'pit_stop_lap', 'pit_stop_milliseconds']].astype(int)

In [50]:
records_data.dtypes

resultid                  object
raceid                    object
driverid                  object
constructorid             object
number                    object
grid                      object
position                  object
positiontext              object
positionorder             object
points                    object
laps                      object
time                      object
milliseconds               int32
fastestlap                 int32
rank                       int32
fastestlaptime            object
fastestlapspeed          float64
statusid                 float64
stop                       int32
pit_stop_lap               int32
pit_stop_time             object
duration                  object
pit_stop_milliseconds      int32
dtype: object

In [51]:
pit_data = records_data[['raceid', 'driverid', 'stop', 'pit_stop_lap', 'pit_stop_time', 'duration', 'pit_stop_milliseconds']]

In [54]:
new_names = {'raceid': 'raceid', 
             'driverid': 'driverid', 
             'stop': 'stop', 
             'pit_stop_lap': 'lap', 
             'pit_stop_time': 'time', 
             'duration': 'duration', 
             'pit_stop_milliseconds': 'milliseconds'}

pit_data.rename(columns=new_names, inplace=True)


C:\Users\jborja\AppData\Local\Temp\ipykernel_26020\3763405722.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pit_data.rename(columns=new_names, inplace=True)


In [55]:
pit_data

,raceid,driverid,stop,lap,time,duration,milliseconds
0,841,39,1,1,17:05:23,26.898,26898
1,841,24,1,1,17:05:23,26.898,26898
2,841,815,1,1,17:05:23,26.898,26898
3,841,155,1,1,17:05:23,26.898,26898
4,841,813,1,1,17:05:23,26.898,26898
...,...,...,...,...,...,...,...
235835,210,55,1,25,None,None,105626
235836,210,21,1,24,None,None,89439
235837,210,56,1,24,None,None,95451
235838,210,30,1,24,None,None,93827


In [56]:
pit_data.to_csv('pit_stops_imputed.csv', index=False)

In [57]:
conn = connection_db()

file_path = os.path.join('pit_stops_imputed.csv')

cursor = conn.cursor()

sql = f"COPY pit_stops FROM STDIN DELIMITER ',' CSV HEADER"
cursor.copy_expert(sql, open(file_path, "r"))
conn.commit()

print(cursor.rowcount, "records inserted.")

235840 records inserted.
